In [1]:
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt

from PhaseFunctionedNetwork import PhaseFunctionedNetwork
from train_utils import train_pfnn_patience
from train_utils import train_pfnn_thresh


# set seeds for reproduceability
torch.manual_seed(42)
np.random.seed(42)
rng = np.random.RandomState(42)

# Style 1 is where the jump style was carefully labelled to blend between jump and idle
# Style 2 is where jumps where just labelled as jumps and idle was labelled as idle

In [2]:
# Process Data 
ROOT_PATH = 'C:/Users/Ana/Desktop/dev/pfnn-dev/Export/v9.0_data/Stylev1/'
INPUT_PATH = 'Input_'
OUTPUT_PATH = 'Output_'

def get_change_in_phase(phase_arr):
    change_in_phase =  phase_arr[1:] - phase_arr[:-1]
    change_in_phase[change_in_phase < 0] = (1.0 - phase_arr[:-1] + phase_arr[1:])[change_in_phase < 0]
    change_in_phase = np.append(change_in_phase,  change_in_phase[-1]) #TODO IF BREAKS append 2pi
    return change_in_phase

def load_data(action_name):
    X_arr = np.float32(np.loadtxt(ROOT_PATH + INPUT_PATH + action_name))
    Y_arr = np.float32(np.loadtxt(ROOT_PATH + OUTPUT_PATH + action_name))
    
    return X_arr, Y_arr

In [3]:
# Load Data
ROUND = 2
# Idle Data
X_idle, Y_idle = load_data('Idle_Static.txt')
P_idle = np.round(np.append(np.linspace(0, 0.99, num=36), np.linspace(0, 0.99, num=36)), ROUND)
delta_idle = get_change_in_phase(P_idle)
print(f"Idle data shape: {X_idle.shape}, {Y_idle.shape}, {P_idle.shape}, {delta_idle.shape}")

# Jump HorizFlat
X_jump_horizflat, Y_jump_horizflat = load_data('Jump_HorizFlat.txt')
P_jump_horizflat = np.round(np.linspace(0, 0.99, num=50), ROUND)
delta_jump_horizflat = get_change_in_phase(P_jump_horizflat)
print(f"Horizflat data shape: {X_jump_horizflat.shape}, {Y_jump_horizflat.shape}, {P_jump_horizflat.shape}, {delta_jump_horizflat.shape}")

# Jump HorizUp
X_jump_horizup, Y_jump_horizup = load_data('Jump_HorizUp.txt')
P_jump_horizup = np.round(np.linspace(0, 0.99, num=48), ROUND)
delta_jump_horizup = get_change_in_phase(P_jump_horizup)
print(f"Horizup data shape: {X_jump_horizup.shape}, {Y_jump_horizup.shape}, {P_jump_horizup.shape}, {delta_jump_horizup.shape}")

# Jump HorizDown
X_jump_horizdown, Y_jump_horizdown = load_data('Jump_HorizDown.txt')
P_jump_horizdown = np.round(np.linspace(0, 0.99, num=56), ROUND)
delta_jump_horizdown = get_change_in_phase(P_jump_horizdown)
print(f"Horizup data shape: {X_jump_horizdown.shape}, {Y_jump_horizdown.shape}, {P_jump_horizdown.shape}, {delta_jump_horizdown.shape}")


Idle data shape: (72, 960), (72, 891), (72,), (72,)
Horizflat data shape: (50, 960), (50, 891), (50,), (50,)
Horizup data shape: (48, 960), (48, 891), (48,), (48,)
Horizup data shape: (56, 960), (56, 891), (56,), (56,)


In [4]:
def preprocess_X(X_arr):
    Xmean, Xstd = X_arr.mean(axis=0), X_arr.std(axis=0)

    # lists to keep track of indices for TRAJECTORY
    X_traj_pos_indices = []
    X_traj_dir_indices = []
    X_traj_style_indices = []
    X_traj_slope_indices = []

    # number of eleements for each trajectory point
    w = 8
    for i in range(0, 95, w):
        X_traj_pos_indices = np.append(X_traj_pos_indices, range(i,i+3)).astype(int)
        X_traj_dir_indices = np.append(X_traj_dir_indices, range(i+3,i+5)).astype(int)
        X_traj_slope_indices = np.append(X_traj_slope_indices, i+5).astype(int)
        X_traj_style_indices = np.append(X_traj_style_indices, range(i+6,i+8)).astype(int)

    # lists to keep track of indices for JOINTS
    X_joint_pos_indices = []
    X_joint_vel_indices = []

    # num of elements for each joint
    w = 6
    for i in range(96, 959, w):
        X_joint_pos_indices = np.append(X_joint_pos_indices, range(i,i+3)).astype(int)
        X_joint_vel_indices = np.append(X_joint_vel_indices, range(i+3,i+6)).astype(int)

    # INPUT Trajectory data
    Xstd[X_traj_pos_indices] = Xstd[X_traj_pos_indices].mean()
    Xstd[X_traj_dir_indices] = Xstd[X_traj_dir_indices].mean()
    Xstd[X_traj_style_indices] = Xstd[X_traj_style_indices].mean()
    Xstd[X_traj_slope_indices] = Xstd[X_traj_slope_indices].mean()

    # INPUT Joint data --> This is where we weight the joints
    # Xstd[X_joint_pos_indices] = Xstd[X_joint_pos_indices].mean() / (joint_weights * 0.1)
    # Xstd[X_joint_vel_indices] = Xstd[X_joint_vel_indices].mean() / (joint_weights * 0.1)
    Xstd[X_joint_pos_indices] = Xstd[X_joint_pos_indices].mean() 
    Xstd[X_joint_vel_indices] = Xstd[X_joint_vel_indices].mean()

    return Xmean, Xstd

def preprocess_Y(Y_arr):
    Ymean, Ystd = Y_arr.mean(axis=0), Y_arr.std(axis=0)

    # PREPROCESS OUTPUT Y
    # lists to keep track of indices for TRAJECTORY
    Y_traj_pos_indices = []
    Y_traj_dir_indices = []

    # number of trajectory elements
    w = 4
    for i in range(0, 23, w): #TODO UPDATE THE RANGE
        Y_traj_pos_indices = np.append(Y_traj_pos_indices, range(i,i+2)).astype(int)
        Y_traj_dir_indices = np.append(Y_traj_dir_indices, range(i+2,i+4)).astype(int)

    # lists to keep track of indices for JOINTS
    Y_joint_pos_indices = []
    Y_joint_vel_indices = []

    # num of joint elements
    w = 6
    for i in range(24, 887, w): #TODO UPDATE THE RANGE
        Y_joint_pos_indices = np.append(Y_joint_pos_indices, range(i,i+3)).astype(int)
        Y_joint_vel_indices = np.append(Y_joint_vel_indices, range(i+3,i+6)).astype(int)

    # OUTPUT Trajectory data
    Ystd[Y_traj_pos_indices] = Ystd[Y_traj_pos_indices].mean()
    Ystd[Y_traj_dir_indices] = Ystd[Y_traj_dir_indices].mean()

    # OUTPUT Joint data --> This is where we weight the joints
    # Ystd[Y_joint_pos_indices] = Ystd[Y_joint_pos_indices].mean() / (joint_weights * 0.1)
    # Ystd[Y_joint_vel_indices] = Ystd[Y_joint_vel_indices].mean() / (joint_weights * 0.1)
    Ystd[Y_joint_pos_indices] = Ystd[Y_joint_pos_indices].mean()
    Ystd[Y_joint_vel_indices] = Ystd[Y_joint_vel_indices].mean()
    
    # translational_vel_mean = (Ystd[-4] + Ystd[-2])/2
    # Ystd[-4] = translational_vel_mean
    # Ystd[-2] = translational_vel_mean

    return Ymean, Ystd

In [5]:
# Concatenate Data into indivudal X, Y, P Arrays
# X = np.concatenate((X_idle, X_jump_horizflat, X_jump_horizup, X_jump_horizdown))
# Y = np.concatenate((Y_idle, Y_jump_horizflat, Y_jump_horizup, Y_jump_horizdown))
# P = np.concatenate((P_idle, P_jump_horizflat, P_jump_horizup, P_jump_horizdown))
# delta_phase = np.concatenate((delta_idle, delta_jump_horizflat, delta_jump_horizup, delta_jump_horizdown))
# Y = np.concatenate([Y, delta_phase [..., np.newaxis]], axis=-1)

X = np.concatenate((X_idle, X_jump_horizflat, X_jump_horizup))
Y = np.concatenate((Y_idle, Y_jump_horizflat, Y_jump_horizup))
P = np.concatenate((P_idle, P_jump_horizflat, P_jump_horizup))
delta_phase = np.concatenate((delta_idle, delta_jump_horizflat, delta_jump_horizup))
Y = np.concatenate([Y, delta_phase [..., np.newaxis]], axis=-1)

# Preprocess Data
Xmean, Xstd = preprocess_X(X)
Ymean, Ystd = preprocess_Y(Y)

WEIGHTS_SAVE_PATH = 'C:/Users/Ana/Desktop/dev/pfnn-dev/unity-pfnn/Assets/Dev/Weights/test/'

for i in range(Xstd.size):
    if (Xstd[i]==0):
        Xstd[i]=1
for i in range(Ystd.size):
    if (Ystd[i]==0):
        Ystd[i]=1

# save means and stds
Xmean.astype(np.float32).tofile(WEIGHTS_SAVE_PATH + 'Xmean.bin')
Ymean.astype(np.float32).tofile(WEIGHTS_SAVE_PATH + 'Ymean.bin')
Xstd.astype(np.float32).tofile(WEIGHTS_SAVE_PATH + 'Xstd.bin')
Ystd.astype(np.float32).tofile(WEIGHTS_SAVE_PATH + 'Ystd.bin')

# normalize data NOTE ORIGINAL DID THIS AFTER SAVING THE MEANS AND STD AS DONE HERE
X = (X - Xmean) / Xstd
Y = (Y - Ymean) / Ystd

# load data for PyTorch training

# append phase as additional feature only for training NN
input = torch.tensor(np.concatenate([X, P [..., np.newaxis]], axis=-1))
target = torch.tensor(Y)

print(f"Input shape {input.shape}")
print(f"Target shape {target.shape}")

dataset = TensorDataset(input, target)

Input shape torch.Size([170, 961])
Target shape torch.Size([170, 892])


In [7]:
# Define PFNN
model = PhaseFunctionedNetwork(input_shape=input.shape[1], output_shape=target.shape[1], dropout=0.7)

# Determine device for training 
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(DEVICE)

# Training variables
BATCH_SIZE = 32
EPOCHS = 500
LR = 0.0001
OPTIMIZER = torch.optim.AdamW(model.parameters(), lr=LR)
train_dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

# Train
model, loss_hist = train_pfnn_patience(model, train_dataloader, optimizer=OPTIMIZER, num_epochs=EPOCHS, device=DEVICE, patience=5)
# model, loss_hist = train_pfnn_thresh(model, train_dataloader, optimizer=OPTIMIZER, num_epochs=EPOCHS, device=DEVICE, threshold=0.05)

# Save
model.precompute_and_save_weights()

100%|██████████| 6/6 [00:00<00:00,  7.49it/s]


Epoch [1/500], Loss: 1.554070477723856


100%|██████████| 6/6 [00:00<00:00, 11.99it/s]


Epoch [2/500], Loss: 1.4961119979927944


100%|██████████| 6/6 [00:00<00:00, 11.96it/s]


...

Epoch [48/500], Loss: 0.23385685051783275


100%|██████████| 6/6 [00:00<00:00, 12.03it/s]


Epoch [49/500], Loss: 0.2761269367246841


100%|██████████| 6/6 [00:00<00:00, 11.94it/s]


Epoch [50/500], Loss: 0.23012328584032482
Training stopped after 5 epochs with no improvement.


In [7]:
# Save loss for later plotting
np.save('v9.0_base_thresh0.01_losshist.npy', loss_hist)

In [8]:
# # Plot
# v9_base = np.load('C:/Users/Ana/Desktop/dev/pfnn-dev/TrainingStats/v9.0_base_losshist.npy')
# v9_withHorizDown = np.load('C:/Users/Ana/Desktop/dev/pfnn-dev/TrainingStats/v9.0_withHorizDown_losshist.npy')

# # Create corresponding x values for each array
# x1 = np.arange(len(v9_base))
# x2 = np.arange(len(v9_withHorizDown))

# # Plot both arrays on the same plot
# plt.plot(x1, v9_base, label='Array 1')
# plt.plot(x2, v9_withHorizDown, label='Array 2')

# # Add title and labels
# plt.title('Training Loss')
# plt.xlabel('Index')
# plt.ylabel('Value')

# # Add a legend
# plt.legend()

# # Display the plot
# plt.show()